# 1. Carga de datos

Cargamos el dataset con el comando load_files desde el directorio del programa.

In [1]:
from sklearn.datasets import load_files
archivos = load_files('txt_sentoken')

# 2. Creación del pipeline

Creamos el pipeline para clasificar las reviews de las películas. Elegimos Naive Bayes como clasificador.

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

clasificador = Pipeline([('vector', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clasif', MultinomialNB(alpha = 1.0))])
clasificador = clasificador.fit(archivos.data, archivos.target)

# 3. Configuración de parámetros - Grid Search

Hacemos una selección de parámetros a variar en el pipeline para el grid search.

In [3]:
parametros = {'vector__ngram_range': [(1, 1), (2, 3)], 'tfidf__use_idf': (True, False), 'clasif__alpha': (1e-2, 1e-3)}

Instanciamos el grid search y evaluamos el grado de exactitud del clasificador segun los parámetros elegidos, el cual es aproximadamente 0.85.

In [4]:
from sklearn.model_selection import GridSearchCV
gridS_clasif = GridSearchCV(clasificador, parametros, cv = 8, n_jobs = -1)
gridS_clasif = gridS_clasif.fit(archivos.data, archivos.target)

gridS_clasif.best_score_

0.8505

Vemos cuáles son, además, los mejores parámetros entre los elegidos.

In [5]:
gridS_clasif.best_params_

{'clasif__alpha': 0.01, 'tfidf__use_idf': False, 'vector__ngram_range': (2, 3)}

# 4. Evaluación del clasificador 

Técnica: Hold Out

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(archivos.data, archivos.target, test_size = 0.75)

print("Cantidad de datos de entrenamiento: ",len(X_train))
print("Cantidad de datos de prueba: ",len(X_test))

Cantidad de datos de entrenamiento:  500
Cantidad de datos de prueba:  1500


In [7]:
clasificador = clasificador.fit(X_train, y_train)

predicted = clasificador.predict(X_test)

import numpy as np
print (np.mean (predicted == y_test))

0.6453333333333333


Técnica: Validación Cruzada

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer

clasif = MultinomialNB()
X = TfidfVectorizer()
X = X.fit_transform(archivos.data)
scores = cross_val_score(clasif, X, archivos.target, cv = 5, error_score = 'raise')

print (scores)                                              

[0.82   0.795  0.8075 0.81   0.7825]


Prueba de diferentes tipos de validación cruzada

    - No estratificada

In [9]:
from sklearn.model_selection import KFold

cv_no_estratificado = KFold(n_splits = 3)
scores = cross_val_score(clasif, X, archivos.target, cv = cv_no_estratificado, error_score = 'raise')

print (scores)

[0.83508246 0.77661169 0.77027027]


    - Estratificada

In [10]:
from sklearn.model_selection import StratifiedKFold

cv_estratificado = StratifiedKFold(n_splits = 3)
scores = cross_val_score(clasif, X, archivos.target, cv = cv_estratificado, error_score = 'raise')

print (scores)

[0.81586826 0.7987988  0.78528529]


    - Aleatoria

In [11]:
from sklearn.model_selection import ShuffleSplit

cv_aleatorio = ShuffleSplit(n_splits = 3)
scores = cross_val_score(clasif, X, archivos.target, cv = cv_aleatorio, error_score = 'raise')

print (scores)

[0.815 0.875 0.825]


# 5. Análisis de los datos obtenidos